# Movie Rating Prediction with DC pipelines -- Demo application

## Intro

This notebook is ment to collect all the scripts and instructions for running the *Movie Rating Prediction* experiment.

Besides the code here you will need:

- Data Civilizer Studio
- (if not integratedin the Studio) the deepER-lite docker image https://hub.docker.com/r/daqcri/deeper-lite/
    - in this case, just attach a bash session to the running image and run the scripts (run.sh and run-pred.sh) from there

### Goal
The goal is to **predict movie ratings** of a generic audience (i.e., *imdb.com* users), given the **critic ratings** and other features (e.g., genre, year) gathered from *rogerebert.com* (a website collecting reviews by critics)

### **For a summary of the results scroll to the end of this notebook**

---


### Data sets
The two employed data sets have the following characteristics:
    
|dataset|#records|notes|
|:--: |:--:|:--:|
|Roger Ebert (https://www.rogerebert.com) |3.5k|contains information about critic rating|
|IMBD (https://www.imdb.com) |6.8k|contains information about audience rating|


**Labeled data for DeepER**:

For running DeepER, labeled data has been provided by manually labeling 370 manually pairs of records (200 of which are matches).

(When trained, DeepER finds 350 matches).


---
Since the scope of the applicaiton is to try DC component, and since imputeDB can only impute numerical values, I selected the attribute year from *roger_ebert* and I randomly injected 20-30 "?" character (on record that I know having a match).

Then, Fahes is employed to discover DMVs.

---

When using ImputeDB for imputing the data, call the rogererbert corresponding file `y_imputeDB.csv`; it will be used to substitute the DMVs.

---

## Preparing data for DeepER

In [7]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import csv
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn import datasets

In [37]:
# paths

base_path = './'
path_in = base_path
path_out = "./"

path_roger = base_path + "data_dump/roger_dump.csv"
path_imdb = base_path + "data_dump/imdb_dump.csv"
lableled = base_path + "labeled_data.csv"

In [15]:
df1 = pd.read_csv(path_roger, encoding = "ISO-8859-1")
df2 = pd.read_csv(path_imdb, encoding = "ISO-8859-1")

df_labeled = pd.read_csv(lableled)

In [16]:
tmp1 = df1
tmp2 = df2

In [17]:
df1[df1.id==3487]
# Red River

,id,movie_name,year,directors,actors,critic_rating,genre,pg_rating,duration
3487,3487,Red River,1948,Howard Hawks,"John Wayne,Montgomery Clift,Joanne Dru Groot,W...",4.0,"Action, Western",Rated NR,133 minutes


In [18]:
df2[df2.id==6882]
# Scarface

,id,movie_name,directors,actors,movie_rating,genre,duration
6882,6882,Scarface,Howard Hawks,"Paul Muni, Ann Dvorak, Karen Morley",7.8,"Action, Crime, Drama",93 min


In [19]:
colSet1 = set(df1.columns)
colSet2 = set(df2.columns)

**We are going to use only the common attributes for DeepER**

- while later, for the data enhancement demo, we are using all the attributes of the records that have matches

In [20]:
commonCols = list(colSet1.intersection(colSet2))
print(commonCols)
colsDeepER = ['id',
 'movie_name',
 'actors',
 #'movie_rating',# this is not the same attrinbute (imdb users vs. critic); we will use that for train/test
 'directors',
 'duration',
 'genre'
]
colsDeepER

['id', 'duration', 'actors', 'movie_name', 'genre', 'directors']


['id', 'movie_name', 'actors', 'directors', 'duration', 'genre']

In [21]:
# these dataset are those to use for the prediction phase of DeepER

df1_deeper = df1[colsDeepER]
df1_deeper.to_csv(path_out+'roger_deeper_prediction.csv',sep=',',index=False, quoting = csv.QUOTE_NONNUMERIC, header=True)

df2_deeper = df2[colsDeepER]
df2_deeper.to_csv(path_out+'imdb_deeper_prediction.csv',sep=',',index=False, quoting = csv.QUOTE_NONNUMERIC, header=True)

### Filtering the set of entities that have been labeled for  DeepER

In [23]:
lableled_ids1 = set(df_labeled['id1'])
lableled_ids2 = set(df_labeled['id2'])

In [24]:
df1_deeper_trainint = df1_deeper[df1_deeper.id.isin(lableled_ids1)]
df2_deeper_trainint = df2_deeper[df2_deeper.id.isin(lableled_ids2)]
print(len(df1_deeper_trainint))
print(len(df2_deeper_trainint))

326
356


In [31]:
df_labled_true = df_labeled[df_labeled['gold']==1][['id1','id2']]

set1 = set(df_labled_true.id1)
set2 = set(df_labled_true.id2)

In [32]:
print(len(set1.difference(lableled_ids1))==0)
print(len(set2.difference(lableled_ids2))==0)

True
True


In [33]:
df_labled_true.to_csv(path_out+'gt.csv',sep=',',index=False, quoting = csv.QUOTE_NONNUMERIC, header=False)

In [34]:
df1_deeper_train = df1_deeper[df1_deeper.id.isin(lableled_ids1)]
df1_deeper_train.to_csv(path_out+'roger_deeper_train.csv',sep=',',index=False, quoting = csv.QUOTE_NONNUMERIC, header=True)

df2_deeper_train = df2_deeper[df2_deeper.id.isin(lableled_ids2)]
df2_deeper_train.to_csv(path_out+'imdb_deeper_train.csv',sep=',',index=False, quoting = csv.QUOTE_NONNUMERIC, header=True)

In [35]:
len(df1_deeper_train)

326

## Steps to be performed with the Data Civilizer modules
Now use DeepER:

The following comments refer to deeper-lite, cloned from github. (Not sure how to set the parameter 0.5, I did runs with also other configurations)

- train the model:
`bash run.sh movies5_repeat roger_deeper_train.csv imdb_deeper_train.csv gt.csv 5 0.5`

- find the matches:
`bash run-pred.sh movies5_repeat roger_deeper_prediction imdb_deeper_prediction 5 C_ER 0 1 no`

## Prediction



In [36]:
#path = "movies5/24Oct_forDeepER/"
#path_ds1 = path + "dump/imdb_dump.csv"
#path_ds2 = path + "dump/roger_dump.csv"

In [40]:
path_year = base_path + "y_imputeDB.csv" # imputeDB (year only)
path_match_deeper = base_path + "predictions_0.csv"
#path_match_deeper = base_path + "predictions_blocking.csv" ## DIFFERENT BLOCKING EMPLOYED
#path_match_deeper = "movies5/match_found_by_deeper.csv"

In [72]:
'''
Piepline mode
'''
mode = 2

**Each of the following "mode" is a different pipeline"
- mode0: set to null DMVs
- mode1: do nothing
- mode2: use imputed values

**re-read the data when change mode, since the datafame might be modified by each of them**

In [73]:
df1 = pd.read_csv(path_roger, encoding = "ISO-8859-1")
df2 = pd.read_csv(path_imdb, encoding = "ISO-8859-1")

df_deeper_match = pd.read_csv(path_match_deeper)
#len(df_deeper_match)
#dpm1=df_deeper_match
#dpm2=df_deeper_match

In [74]:
if mode == 0:
    df1=df1[~(df1.year=="?")]
elif mode == 1:
    #continue
    df_y = pd.read_csv(path_year, encoding = "ISO-8859-1")
elif mode==2:
    df_y = pd.read_csv(path_year, encoding = "ISO-8859-1")
    df1.year = df_y['roger_ebert3_y_nulls.year']

In [75]:
# join the two tables (using the metches)
partial = pd.merge(df2, df_deeper_match, left_on = 'id', right_on = 'id2')
df_ = pd.merge(df1, partial, left_on = 'id', right_on = 'id1')

In [76]:
colnames = ['id_x', 'movie_name_x', 'year', 'directors', 'aa',
       'critic_rating', 'genre', 'pg_rating', 'duration', 'id_y',
       'movie_name_y', 'directorss', 'actors', 'movie_rating', 'genres',
       'dd', 'id1', 'id2']

In [77]:
df_.columns=colnames
df_join = df_.drop(['id1', 'id2'], axis=1)

In [78]:
# fueature selected
cc = [
    'critic_rating', # from roger
    'pg_rating',
    'year',
    'duration',
    'actors',
    'movie_rating' # from imdb
     ]

In [79]:
movies = df_join[cc]
movies = movies.dropna()

In [80]:
X = movies.drop('movie_rating',axis=1)
Y = movies[['movie_rating']]

X = pd.get_dummies(X)

In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=1)

In [82]:
from sklearn.linear_model import LinearRegression

regression_model = LinearRegression()
regression_model.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [83]:
regression_model.score(X_test, Y_test)

0.4896973467886723

In [84]:
import numpy
numpy.corrcoef(movies['movie_rating'], movies['critic_rating'])

array([[1.       , 0.4745882],
       [0.4745882, 1.       ]])

In [85]:
0.4745882

0.4745882

In [86]:
numpy.corrcoef(movies['movie_rating'], movies['critic_rating'])

array([[1.       , 0.4745882],
       [0.4745882, 1.       ]])

## Eperiments with blocking (candidate pair generation)

In [87]:
nx_res = list(df_join["movie_name_x"].values)
ny_res = list(df_join["movie_name_y"].values)

nx = list(df1["movie_name"].values)
ny = list(df2["movie_name"].values)

In [88]:
import string
table = str.maketrans({key: None for key in string.punctuation})

k1 = list(map(lambda x: "".join(x.lower().strip().translate(table).split()[0:2]), nx_res))
k2 = list(map(lambda x: "".join(x.lower().strip().translate(table).split()[0:2]), ny_res))

In [89]:
pp = list(zip(k1,k2))

In [90]:
counter = 0
for p in pp:
    if p[0]!=p[1]:
        counter+=1
        print(p[0]+" "+p[1])
print(counter)

22jump 21jump
21jump 22jump
thebaadermeinhof thebaader
3


In [91]:
k1 = list(map(lambda x: "".join(x.lower().strip().translate(table).split()[0:2]), nx))
k2 = list(map(lambda x: "".join(x.lower().strip().translate(table).split()[0:2]), ny))

In [92]:
dfk1 = df1
dfk2 = df2

dfk1['k'] = k1
dfk2['k'] = k2

In [93]:
blockCandidates = pd.merge(dfk1, dfk2, left_on = 'k', right_on = 'k')
candidates_blocking = blockCandidates[["id_x","id_y"]]

In [94]:
candidates_blocking.to_csv(path_out+'blocking_candidates.csv',sep=',',index=False, quoting = csv.QUOTE_NONNUMERIC, header=False)

In [95]:
len(candidates_blocking)

2412

# Result presentation

## Pearson correlation
The initial **hypothesis** was to verify if there is a correlation between user rating and critic rating.

The only way to do that is to perform entity resolution between the two datasets.

#### results:
- **Pearson correlatoin**: 0.47

## Linear Regression (prediciton)

- **Metrics**: r squared ([r^2](https://en.wikipedia.org/wiki/Coefficient_of_determination)) is employed for assessing the quality of the predictions.
The ideal method reaches 1.

- Train/Test: 75/25

---

**Results**:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-0lax"></th>
    <th class="tg-0lax">Pipeline</th>
    <th class="tg-0lax">r^2</th>
  </tr>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">DeepER</td>
    <td class="tg-0lax">0.18</td>
  </tr>
  <tr>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">Fahes(set null)+DeepER</td>
    <td class="tg-0lax">0.40</td>
  </tr>
  <tr>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">Fahes+ImputeDB+DeepER</td>
    <td class="tg-0lax">0.50</td>
  </tr>
</table>

In [96]:
#dpmSet1 = set(zip(list(dpm1.id1.values),list(dpm1.id2.values)))
#dpmSet2 = set(zip(list(dpm2.id1.values),list(dpm2.id2.values)))